In [58]:
from IPython.display import display
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import spacy

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)

%matplotlib inline


In [59]:
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

wl = WordNetLemmatizer()
snow = SnowballStemmer('english')


[nltk_data] Downloading package punkt to /home/mikes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mikes/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/mikes/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mikes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
 %cat '/home/mikes/nltk_data/corpora/stopwords/english'

mg
mgs
milligram
milligrams
i
me
my
myself
we
our
ours
ourselves
you
you're
you've
you'll
you'd
your
yours
yourself
yourselves
he
him
his
himself
she
she's
her
hers
herself
it
it's
its
itself
they
them
their
theirs
themselves
what
which
who
whom
this
that
that'll
these
those
am
is
are
was
were
be
been
being
have
has
had
having
do
does
did
doing
a
an
the
and
but
if
or
because
as
until
while
of
at
by
for
with
about
against
between
into
through
during
before
after
above
below
to
from
up
down
in
out
on
off
over
under
again
further
then
once
here
there
when
where
why
how
all
any
both
each
few
more
most
other
some
such
no
nor
not
only
own
same
so
than
too
very
s
t
can
will
just
don
don't
should
should've
now
d
ll
m
o
re
ve
y
ain
aren
aren't
couldn
couldn't
didn
didn't
doesn
doesn't
hadn
hadn't
hasn
hasn't
haven
haven't
isn
isn't
ma
mightn
mightn't
mustn
mustn't
needn
needn't
shan
shan't
shouldn
shouldn't
wasn
wasn't
weren
weren't
won
won't
wouldn
wouldn't


In [61]:
dir = "/home/mikes/Documents/jupyterEnvironments/VAERS_analysis/"
file = "VAERS data deaths only Edited for NLP.csv"
#file = "VAERS data deaths only Edited for NLP for HAD COVID.csv"
path = os.path.join(dir, file)

vd = pd.read_csv(path, 
                 delimiter = ",", 
                 encoding = "ISO-8859-1")
vd.columns

Index(['VAERS_ID', 'SYMPTOM_TEXT_TARGET', 'HOSPICE_TARGET', 'RECVDATE',
       'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX', 'RPT_DATE',
       'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL',
       'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE',
       'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS',
       'OTHER_MEDS_TARGET', 'CUR_ILL', 'CUR_ILL_TARGET', 'HISTORY',
       'HISTORY_TARGET', 'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE',
       'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'],
      dtype='object')

In [188]:
#convert to lowercase and remove punctuations and characters and then strip
def preprocess(text):
    text = text.lower() #lowercase text
    text=text.strip()  #get rid of leading/trailing whitespace 
    text=re.compile('<.*?>').sub('', text) #Remove HTML tags/markups
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  #Replace punctuation with space. Careful since punctuation can sometime be useful
    text = re.sub('\s+', ' ', text)  #Remove extra space and tabs
    text = re.sub(r'\[[0-9]*\]',' ',text) #[0-9] matches any digit (0 to 10000...)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) #matches any digit from 0 to 100000..., \D matches non-digits
    text = re.sub(r'\s+',' ',text) #\s matches any whitespace, \s+ matches multiple whitespace, \S matches non-whitespace 
    
    return text

#1a. STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#1b. STOPWORD REMOVAL
def stopword_medications(string):
    """
    Additional stopwords specific to medications added to text file "/home/mikes/nltk_data/corpora/stopwords/english.medications"
    """
    a= [i for i in string.split() if i not in stopwords.words('english.medications')]
    return ' '.join(a)

#2. STEMMING 
# Initialize the stemmer
#snow = SnowballStemmer('english')
def stemming(string):
    a=[snow.stem(i) for i in word_tokenize(string) ]
    return " ".join(a)

#3. LEMMATIZATION
# Initialize the lemmatizer
#wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    string = str(string)
    return lemmatizer(stopword(preprocess(string)))
#    return lemmatizer(preprocess(string))

In [63]:
#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.

The support is the number of occurrences of each class in y_true.

If pos_label is None and in binary classification, this function returns the average precision, recall and F-measure if average is one of 'micro', 'macro', 'weighted' or 'samples'.

Read more in the [User Guide](https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics).

In [64]:

def logistic_regression_tfidf(df_train, df_test, ID, TEXT, TARGET, perc_cutoff = 80):
    if perc_cutoff > 1: perc_cutoff = perc_cutoff/100
    df_train = df_train.loc[df_train[TARGET].notnull(), [ID, TEXT, TARGET ]]
    df_train['word_count'] = df_train[TEXT].apply(lambda x: len(str(x).split()))
    #print(df_train[df_train[TARGET]==1]['word_count'].mean()) #Positive match
    #print(df_train[df_train[TARGET]==0]['word_count'].mean()) #Not positive match 
    df_train['clean_text'] = df_train[TEXT].apply(lambda x: finalpreprocess(x))
    # create Word2vec model
    #  here words_f should be a list containing words from each document. say 1st row of the list is words from the 1st document/sentence
    #  length of words_f is number of documents/sentences in your dataset
    df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']] #convert preprocessed sentence to tokenized sentence
    model = Word2Vec(df_train['clean_text_tok'],min_count=1)  #min_count=1 means word should be present at least across all documents,
    #if min_count=2 means if the word is present less than 2 times across all the documents then we shouldn't consider it
    w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))  #combination of word and its vector

    #SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
    # Input: "reviewText", "rating" and "time"
    # Target: "log_votes"
    X_train, X_val, y_train, y_val = train_test_split(df_train["clean_text"],
                                                      df_train[TARGET],
                                                      test_size=0.2,
                                                      shuffle=True)
    X_train_tok= [nltk.word_tokenize(i) for i in X_train]  #for word2vec
    X_val_tok= [nltk.word_tokenize(i) for i in X_val]      #for word2vec
    #TF-IDF
    # Convert x_train to vector since model can only run on numbers and not words- Fit and transform
    tfidf_vectorizer = TfidfVectorizer(use_idf=True)
    X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) #tfidf runs on non-tokenized sentences unlike word2vec
    # Only transform x_test (not fit and transform)
    X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) #Don't fit() your TfidfVectorizer to your test data: it will 
    #change the word-indexes & weights to match test data. Rather, fit on the training data, then use the same train-data-
    #fit model on the test data, to reflect the fact you're analyzing the test data only based on what was learned without 
    #it, and the have compatible

    ######################################################################
    
    #FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
    lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
    lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

    #Predict y value for test dataset
    y_predict = lr_tfidf.predict(X_val_vectors_tfidf)
    y_prob = lr_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]


    print(classification_report(y_val,y_predict))
    print('Confusion Matrix:',confusion_matrix(y_val, y_predict))

    fpr, tpr, thresholds = roc_curve(y_val, y_prob)
    roc_auc = auc(fpr, tpr)
    print('AUC:', roc_auc)

    ### EXAMINE TEST SET WITH MODEL. This includes the data from training (both pos and neg targets) ###
    df_test['clean_text'] = vd[TEXT].apply(lambda x: finalpreprocess(x)) #preprocess the data
    X_test=df_test['clean_text'] 
    X_vector=tfidf_vectorizer.transform(X_test) #converting X_test to vector
    y_predict = lr_tfidf.predict(X_vector)      #use the trained model on X_vector
    y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
    df_test['predict_prob']= y_prob
    df_test['target']= y_predict
    # display(df_test[['VAERS_ID','SYMPTOM_TEXT','predict_prob','target']].head())
    # #final=df_test[['id','target']].reset_index(drop=True)
    # #final.to_csv('submission.csv')
    df_results = df_test[(
                            (df_test["target"] == 1) & 
                            (df_test["predict_prob"] >= perc_cutoff)
    )]
    num_found = df_results.shape[0]
    num_total = df_test.shape[0]
    print("{}/{} ({})".format(num_found, num_total, num_found/num_total))
    return num_found,df_results



In [65]:
def run_analysis(df, ID, TEXT, TARGET, iterations = 10):
    results = []
    for i in range(0,iterations,1):
        print(); print(" Iteration {} {} ==========".format(i, TEXT))
        df_train = df.loc[vd[TARGET].notnull(), [ID,TEXT,TARGET]]
        df_train['word_count'] = df_train[TEXT].apply(lambda x: len(str(x).split()))
        print("Average word count, positive targets: {}".format(df_train[df_train[TARGET]==1]['word_count'].mean())) #Has covid
        print("Average word count, negative targets: {}".format(df_train[df_train[TARGET]==0]['word_count'].mean())) #Has covid
        num_found,df_results = logistic_regression_tfidf(df_train, df, ID,TEXT,TARGET)
        results.append((df_results, num_found, num_found/df.shape[0]))

    for i in results:
        print(i[1], i[2])
    
    return results


In [202]:
def parse_out_meds(df):
    def parse_meds_from_phrase(phrase): # Takes an iterable of strings
        # Parse out the vitamins first, since they con be single or multiple words
        p1 = "([multi]*vit[amins]*\s*[abcdek]{1}\d?( |$))" # I.e. "vitamin c" or "multivitamin d3", "vitamins b", etc.
        p2 = "(([multi]*vit[amins]*\s*)?thiamin|riboflavin|niacin|(pantothenic\s*acid)|pyridoxal|cobalamin|biotin|folate|(folic\s*acid))" # I.e. "vitamin(s) thiamin"
        p3 = "([multi]*vit[amins]*( |$))" # "<multi>vitamin<s>" only
        p = '|'.join([p1, p2, p3])

        m = re.findall(p, phrase, re.IGNORECASE)
        # Remove any contiguous strings of quotes, brackets, or parens. Leave commas and spaces
        m = re.sub("[\'\"\]\[\(\)]*", "", str(m)) 
        # Split by commas, and remove words  that are all spaces. Leaves spaces between words
        m = [w.strip() for w in m.split(",") if not re.match("^\s*$", w) ] 
        meds_list = list(set(m)) # remove duplicate strings

        # Remove all the found vitamins from phrase
        for vit in m:
            phrase = phrase.replace(vit, "")

        # Turn remaining meds into set
        phrase = " ".join(phrase.split()) # remove blocks of contiguous whitespaces
        phrase = list(set(phrase.split(" "))) # Set to remove dups, then back to a list for adding

        # Add together
        meds_list += phrase

        return sorted(meds_list, key=str.lower)
        
    s = df["OTHER_MEDS"].apply(lambda x: preprocess(str(x)))
    s = s.apply(stopword_medications)
    for phrase in s: 
        result = parse_meds_from_phrase(phrase)
        print(result)

parse_out_meds(vd)


['alphagan', 'chlorthlidone', 'chondrotin', 'colocrys', 'gabapentin', 'glusosamine', 'lanaoprost', 'lantus', 'lisinopril', 'metoprol', 'needed', 'novolog', 'solostar', 'tartrate']
['fentanyl', 'hydrocodone', 'mertazipine', 'quietapine']
['amitizide', 'carbidoba', 'chloride', 'clonazepam', 'ellipta', 'escalitapram', 'fentanyl', 'incruse', 'levodopa', 'levothyroxine', 'mirtazipane', 'morphine', 'potassium', 'prednisone', 'sulfate', 'torsemide', 'trazadone']
['benaprezil', 'chloride', 'glipizide', 'latanoprost', 'levothyroxine', 'metformin', 'metoprolol', 'potassium', 'pravastatin', 'tartrate']
['bid', 'capsule', 'cogentin', 'dyazide', 'hctz', 'invega', 'lipitor', 'lithium', 'months', 'norvasc', 'oxybutynin', 'q', 'qd', 'qhs', 'qid', 'trenza', 'triamterene']
['acid', 'albuteral', 'amlodipine', 'carvedilol', 'chloride', 'ipratropium', 'levetiracetam', 'olanzapine', 'sodium', 'sulfate', 'symbicort', 'valproic']
['acid', 'albuterol', 'asa', 'benadryl', 'colace', 'ferrous', 'folic acid', 'gab

['nan']
['nan']
['acido', 'ferroso', 'folico', 'keppra', 'lamictal', 'lorazepam', 'motrin', 'multivitamínico', 'quetiapina', 'sultato', 'vimpat', 'vitamina k', 'zyprexa']
['apixaban', 'feosol', 'folic acid', 'levothyroxine', 'lorazapam', 'metoprolol', 'natural', 'pravastatin', 'release', 'sodium', 'succinate']
['flexeril', 'keppra', 'vitamins']
['acetaminophen', 'benzonatate', 'clopidogrel', 'cyclobenzaprine', 'docusate', 'doxylamine', 'escitalopram', 'fluticasone', 'furosemide', 'hydrocortisone', 'latanoprost', 'levothyroxine', 'metformin', 'midodrine', 'nitroglycerin', 'ophthalmic', 'prn', 'senna', 'topical']
['atorvastatin', 'day', 'fluoxetine', 'lorazepam', 'megace', 'pepcid', 'quetiapine', 'temazepam', 'tentral']
['deforming', 'metropol', 'quetapil', 'sereoquel', 'simvastatina', 'trazodone']
['nan']
['aspiri', 'carvedilol', 'chondroitin', 'complex', 'day', 'glucosamine', 'lutein', 'metformin', 'multivitamin', 'tablet', 'trunature', 'vision', 'zeaxanthin', '½']
['diabetes', 'heartb

['nan']
['father', 'medications', 'sure']
['thyroid']
['asa', 'atorvastatin', 'carvedilol', 'colchicine', 'e', 'eliquis', 'fenofibrate', 'folic acid', 'glimepiride', 'lisinopril', 'proscar', 'tamsulosin', 'thiamin', 'tramadol']
['nan']
['blood', 'report', 'thinner', 'unknown', 'use', 'vaccination', 'visit']
['lyrica']
['unknown']
['nan']
['humera', 'iron', 'multi', 'prednisone', 'vitamin']
['albuterol', 'anoro', 'clonazepam', 'co', 'ellipta', 'er', 'estradiol', 'fish', 'magnesium', 'metoprolol', 'oil', 'pravastatin', 'prednisone', 'probiotic', 'q', 'vit', 'vit b', 'vit c', 'zyrtec']
['singulair']
['allopurinol', 'amiodarone', 'aspirin', 'atorvastatin', 'capsule', 'carvedilol', 'clopidogrel', 'folic acid', 'hctz', 'humalog', 'levemir', 'losartan', 'omeprazole', 'p', 'tablet']
['blood', 'hypertension', 'thinning', 'thyroid']
['apixaban', 'aspirin', 'cellcept', 'cyclosporine', 'dexamethasone']
['omeprazole', 'symbicort']
['ferrous', 'omeprazole', 'sulfate']
['nan']
['nan']
['nan']
['anuso

In [67]:
meds_list = []
def parse_meds(t):
    try: _l = t.split(",")
    except: return
    for i in _l:
        if i not in meds_list:
            meds_list.append(i)
vd["OTHER_MEDS"].apply(parse_meds)
print(meds_list)

['Lisinopril', ' Novolog', '', ' Lantus Solostar', ' Gabapentin', ' Glusosamine-chondrotin', ' Lanaoprost', ' Alphagan', ' Metoprol Tartrate', '  Colocrys. . as needed', ' Chlorthlidone.', 'Quietapine', ' Mertazipine', ' Hydrocodone', ' Fentanyl', 'Torsemide', ' Trazadone', ' Amitizide', ' Morphine Sulfate', ' Carbidoba-Levodopa', ' Prednisone', ' Potassium Chloride', ' Escalitapram', ' Incruse Ellipta', ' Mirtazipane', ' Clonazepam', ' Levothyroxine', 'Metoprolol-Tartrate', ' Latanoprost', ' Glipizide', ' Pravastatin', ' Metformin', ' Benaprezil', ' Potassium-Chloride', '- Invega Trenza 546mg Q3months  - Lithium 300mg BID - Oxybutynin 5mg QID - lipitor 20mg QHS  - cogentin 2mg BID  - Norvasc 5mg QD - Dyazide Capsule 37.5-25 MG (Triamterene-HCTZ) QD', 'Albuteral Sulfate Amlodipine Carvedilol Ipratropium Levetiracetam Olanzapine Sodium Chloride Symbicort Valproic Acid', 'Mucomyst ASA Benadryl Periactin Colace Ferrous Sulfate Folic Acid Gabapentin Sennosides Albuterol Ipratropium Melaton